<a href="https://colab.research.google.com/github/Aleeg10/Quant/blob/main/Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# ==========================
# 1. Descargar datos
# ==========================
symbols = ['AAPL', 'SPY']
data = yf.download(symbols, start='2020-01-01', end='2024-12-31', auto_adjust=True)['Close']

# ==========================
# 2. Calcular SMA y retornos
# ==========================
df = pd.DataFrame()
df['AAPL_Close'] = data['AAPL']
df['SPY_Close'] = data['SPY']
df['SMA_20'] = df['AAPL_Close'].rolling(window=20).mean()
df['Returns'] = df['AAPL_Close'].pct_change()
df = df.dropna()

# ==========================
# 3. Crear señal de momentum
# ==========================
df['Signal'] = np.where(df['AAPL_Close'] > df['SMA_20'], 1, -1)
df['Strategy_Returns'] = df['Returns'] * df['Signal'].shift(1)

# ==========================
# 4. Retornos acumulados
# ==========================
df['Buy_Hold'] = (1 + df['Returns']).cumprod()
df['Momentum_Strategy'] = (1 + df['Strategy_Returns']).cumprod()
df['SPY'] = (1 + df['SPY_Close'].pct_change().fillna(0)).cumprod()

# ==========================
# 5. Puntos de entrada/salida
# ==========================
df['Signal_Change'] = df['Signal'].diff()
entry_points = df[df['Signal_Change'] == 2]  # -1 -> +1
exit_points = df[df['Signal_Change'] == -2]  # +1 -> -1

# ==========================
# 6. Gráfico interactivo
# ==========================
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['Buy_Hold'],
                         mode='lines', name='Buy & Hold AAPL',
                         line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index, y=df['Momentum_Strategy'],
                         mode='lines', name='Momentum Strategy',
                         line=dict(color='orange')))
fig.add_trace(go.Scatter(x=df.index, y=df['SPY'],
                         mode='lines', name='SPY Benchmark',
                         line=dict(color='green')))

# Marcar entradas y salidas
fig.add_trace(go.Scatter(x=entry_points.index, y=entry_points['Momentum_Strategy'],
                         mode='markers', name='Entradas',
                         marker=dict(symbol='triangle-up', color='green', size=10)))
fig.add_trace(go.Scatter(x=exit_points.index, y=exit_points['Momentum_Strategy'],
                         mode='markers', name='Salidas',
                         marker=dict(symbol='triangle-down', color='red', size=10)))

fig.update_layout(title='Momentum Strategy vs Buy & Hold vs SPY',
                  xaxis_title='Fecha', yaxis_title='Cumulative Returns',
                  template='plotly_dark')
fig.show()


[*********************100%***********************]  2 of 2 completed


In [ ]:
# prompt: print results

print("Buy & Hold Cumulative Return:", df['Buy_Hold'].iloc[-1])
print("Momentum Strategy Cumulative Return:", df['Momentum_Strategy'].iloc[-1])
print("SPY Benchmark Cumulative Return:", df['SPY'].iloc[-1])

Buy & Hold Cumulative Return: 3.2082232244322535
Momentum Strategy Cumulative Return: 2.8387514106969083
SPY Benchmark Cumulative Return: 1.9362071748439804
